In [ ]:
import sys
print(sys.executable)
print(sys.version)

from pymongo import MongoClient
from datetime import datetime
import pandas as pd

# Establish connection to MongoDB
# Ensure MongoDB is running on localhost:27017
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db'] # Connect to or create the 'eduhub_db' database

print("Connected to MongoDB. Database: eduhub_db")

# --- Create collections with validation rules ---

# 1. users collection
# Stores information about students and instructors
try:
    db.create_collection(
        'users',
        validator={
            '$jsonSchema': {
                'bsonType': 'object',
                'required': ['_id', 'username', 'email', 'password_hash', 'role', 'created_at', 'updated_at'],
                'properties': {
                    '_id': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required (e.g., UUID)'
                    },
                    'username': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required',
                        'minLength': 3
                    },
                    'email': {
                        'bsonType': 'string',
                        'description': 'must be a string and a valid email format, and is required',
                        'pattern': '^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$'
                    },
                    'password_hash': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required'
                    },
                    'role': {
                        'bsonType': 'string',
                        'description': 'must be "student" or "instructor" and is required',
                        'enum': ['student', 'instructor']
                    },
                    'created_at': {
                        'bsonType': 'date',
                        'description': 'must be a date and is required'
                    },
                    'updated_at': {
                        'bsonType': 'date',
                        'description': 'must be a date and is required'
                    }
                }
            }
        }
    )
    print("Collection 'users' created with validation.")
except Exception as e:
    print(f"Error creating 'users' collection or it already exists: {e}")

# 2. courses collection
# Stores information about courses offered
try:
    db.create_collection(
        'courses',
        validator={
            '$jsonSchema': {
                'bsonType': 'object',
                'required': ['_id', 'title', 'description', 'instructor_id', 'created_at', 'updated_at'],
                'properties': {
                    '_id': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required (e.g., UUID)'
                    },
                    'title': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required',
                        'minLength': 5
                    },
                    'description': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required',
                        'minLength': 10
                    },
                    'instructor_id': {
                        'bsonType': 'string',
                        'description': 'must be a string referencing users._id and is required'
                    },
                    'created_at': {
                        'bsonType': 'date',
                        'description': 'must be a date and is required'
                    },
                    'updated_at': {
                        'bsonType': 'date',
                        'description': 'must be a date and is required'
                    }
                }
            }
        }
    )
    print("Collection 'courses' created with validation.")
except Exception as e:
    print(f"Error creating 'courses' collection or it already exists: {e}")

# 3. enrollments collection
# Records student enrollments in courses
try:
    db.create_collection(
        'enrollments',
        validator={
            '$jsonSchema': {
                'bsonType': 'object',
                'required': ['_id', 'student_id', 'course_id', 'enrollment_date', 'status'],
                'properties': {
                    '_id': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required (e.g., UUID)'
                    },
                    'student_id': {
                        'bsonType': 'string',
                        'description': 'must be a string referencing users._id and is required'
                    },
                    'course_id': {
                        'bsonType': 'string',
                        'description': 'must be a string referencing courses._id and is required'
                    },
                    'enrollment_date': {
                        'bsonType': 'date',
                        'description': 'must be a date and is required'
                    },
                    'status': {
                        'bsonType': 'string',
                        'description': 'must be "active", "completed", or "dropped" and is required',
                        'enum': ['active', 'completed', 'dropped']
                    }
                }
            }
        }
    )
    print("Collection 'enrollments' created with validation.")
except Exception as e:
    print(f"Error creating 'enrollments' collection or it already exists: {e}")

# 4. lessons collection
# Details individual lessons within courses
try:
    db.create_collection(
        'lessons',
        validator={
            '$jsonSchema': {
                'bsonType': 'object',
                'required': ['_id', 'course_id', 'title', 'content', 'order'],
                'properties': {
                    '_id': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required (e.g., UUID)'
                    },
                    'course_id': {
                        'bsonType': 'string',
                        'description': 'must be a string referencing courses._id and is required'
                    },
                    'title': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required',
                        'minLength': 5
                    },
                    'content': {
                        'bsonType': 'string',
                        'description': 'must be a string containing lesson material and is required',
                        'minLength': 20
                    },
                    'order': {
                        'bsonType': 'int',
                        'description': 'must be an integer representing lesson order and is required',
                        'minimum': 1
                    }
                }
            }
        }
    )
    print("Collection 'lessons' created with validation.")
except Exception as e:
    print(f"Error creating 'lessons' collection or it already exists: {e}")

# 5. assignments collection
# Manages course assignments
try:
    db.create_collection(
        'assignments',
        validator={
            '$jsonSchema': {
                'bsonType': 'object',
                'required': ['_id', 'lesson_id', 'title', 'description', 'due_date'],
                'properties': {
                    '_id': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required (e.g., UUID)'
                    },
                    'lesson_id': {
                        'bsonType': 'string',
                        'description': 'must be a string referencing lessons._id and is required'
                    },
                    'title': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required',
                        'minLength': 5
                    },
                    'description': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required',
                        'minLength': 10
                    },
                    'due_date': {
                        'bsonType': 'date',
                        'description': 'must be a date and is required'
                    }
                }
            }
        }
    )
    print("Collection 'assignments' created with validation.")
except Exception as e:
    print(f"Error creating 'assignments' collection or it already exists: {e}")

# 6. submissions collection
# Stores student submissions for assignments
try:
    db.create_collection(
        'submissions',
        validator={
            '$jsonSchema': {
                'bsonType': 'object',
                'required': ['_id', 'assignment_id', 'student_id', 'submission_date', 'content'],
                'properties': {
                    '_id': {
                        'bsonType': 'string',
                        'description': 'must be a string and is required (e.g., UUID)'
                    },
                    'assignment_id': {
                        'bsonType': 'string',
                        'description': 'must be a string referencing assignments._id and is required'
                    },
                    'student_id': {
                        'bsonType': 'string',
                        'description': 'must be a string referencing users._id and is required'
                    },
                    'submission_date': {
                        'bsonType': 'date',
                        'description': 'must be a date and is required'
                    },
                    'content': {
                        'bsonType': 'string',
                        'description': 'must be a string representing submission content (e.g., text, file path) and is required',
                        'minLength': 1
                    },
                    'grade': {
                        'bsonType': ['double', 'null'], # Allow float or null
                        'description': 'must be a number between 0 and 100 if present',
                        'minimum': 0,
                        'maximum': 100
                    },
                    'feedback': {
                        'bsonType': ['string', 'null'], # Allow string or null
                        'description': 'must be a string containing feedback if present'
                    }
                }
            }
        }
    )
    print("Collection 'submissions' created with validation.")
except Exception as e:
    print(f"Error creating 'submissions' collection or it already exists: {e}")

# Close the connection
client.close()
print("MongoDB connection closed.")

In [23]:
!pip install pymongo

Defaulting to user installation because normal site-packages is not writeable


In [9]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import uuid # Used for generating unique string _id values
import random # For randomizing data

# Establish connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

print("Connected to MongoDB. Inserting sample documents...")

# Clear existing data to ensure a fresh start each time you run the script
# This is useful for development but should be used with caution in production!
try:
    db.users.delete_many({})
    db.courses.delete_many({})
    db.enrollments.delete_many({})
    db.lessons.delete_many({})
    db.assignments.delete_many({})
    db.submissions.delete_many({})
    print("Cleared existing data from collections.")
except Exception as e:
    print(f"Error clearing collections: {e}")

# --- 1. Sample Users Collection Documents ---
sample_users = []
instructor_ids = []
student_ids = []

for i in range(20):
    user_id = str(uuid.uuid4())
    role = "student" if i % 2 == 0 else "instructor" # Alternate roles
    
    if role == "instructor":
        instructor_ids.append(user_id)
        first_name = random.choice(["Alice", "Bob", "Charlie", "Diana", "Eve"])
        last_name = random.choice(["Smith", "Jones", "Williams", "Brown", "Davis"])
        bio = f"Experienced instructor in {random.choice(['AI', 'Web Dev', 'Data Science', 'Networking'])}."
        skills = random.sample(["Python", "Java", "C++", "JavaScript", "MongoDB", "SQL", "Machine Learning", "Cloud Computing"], k=random.randint(2,4))
    else:
        student_ids.append(user_id)
        first_name = random.choice(["Frank", "Grace", "Heidi", "Ivan", "Judy", "Karl", "Linda", "Mike", "Nancy", "Oscar"])
        last_name = random.choice(["Wilson", "Miller", "Taylor", "Anderson", "Thomas", "Jackson", "White", "Harris", "Martin", "Thompson"])
        bio = f"Eager to learn about {random.choice(['programming', 'design', 'history', 'science'])}."
        skills = random.sample(["Beginner", "Intermediate", "Fast Learner", "Problem Solver"], k=random.randint(1,2))

    sample_users.append({
        "_id": user_id,
        "username": f"{first_name.lower()}_{last_name.lower()}{i}",
        "email": f"{first_name.lower()}.{last_name.lower()}{i}@example.com",
        "password_hash": f"hashed_password_{uuid.uuid4().hex[:8]}",
        "role": role,
        "created_at": datetime.utcnow() - timedelta(days=random.randint(30, 365)),
        "updated_at": datetime.utcnow() - timedelta(days=random.randint(1, 30)),
        "first_name": first_name,
        "last_name": last_name,
        "profile": {
            "bio": bio,
            "avatar": f"https://placehold.co/100x100/{random.choice(['ADD8E6', 'FFDDC1', 'D4A6C5'])}/000000?text={first_name[0]}{last_name[0]}",
            "skills": skills
        },
        "is_active": random.choice([True, False])
    })

try:
    db.users.insert_many(sample_users)
    print(f"Inserted {len(sample_users)} sample user documents.")
except Exception as e:
    print(f"Error inserting sample user documents: {e}")

# --- 2. Sample Courses Collection Documents ---
sample_courses = []
course_ids = []
categories = ["Programming", "Web Development", "Data Science", "Design", "Business", "Marketing", "Science", "Arts"]
levels = ["beginner", "intermediate", "advanced"]

for i in range(8):
    course_id = str(uuid.uuid4())
    course_ids.append(course_id)
    
    sample_courses.append({
        "_id": course_id,
        "title": f"{random.choice(['Mastering', 'Introduction to', 'Advanced', 'Fundamentals of'])} {random.choice(categories)} Course {i+1}",
        "description": f"A comprehensive course covering {random.choice(categories).lower()} concepts and practices. This course is for {levels[i % 3]} learners.",
        "instructor_id": random.choice(instructor_ids), # Random instructor
        "category": random.choice(categories),
        "level": levels[i % 3],
        "duration": random.randint(20, 100), # hours
        "price": round(random.uniform(29.99, 299.99), 2),
        "tags": random.sample(["Online", "Certification", "Project-based", "Interactive", "Self-paced", "Beginner Friendly"], k=random.randint(2,4)),
        "created_at": datetime.utcnow() - timedelta(days=random.randint(60, 180)),
        "updated_at": datetime.utcnow() - timedelta(days=random.randint(7, 60)),
        "is_published": random.choice([True, True, True, False]) # Mostly published
    })

try:
    db.courses.insert_many(sample_courses)
    print(f"Inserted {len(sample_courses)} sample course documents.")
except Exception as e:
    print(f"Error inserting sample course documents: {e}")

# --- 3. Sample Enrollments Collection Documents ---
sample_enrollments = []
enrollment_statuses = ["active", "completed", "dropped"]

# Ensure all students are enrolled in at least one course, and then add more
for i in range(15):
    student_id = random.choice(student_ids)
    course_id = random.choice(course_ids)
    
    sample_enrollments.append({
        "_id": str(uuid.uuid4()),
        "student_id": student_id,
        "course_id": course_id,
        "enrollment_date": datetime.utcnow() - timedelta(days=random.randint(10, 90)),
        "status": random.choice(enrollment_statuses)
    })

try:
    db.enrollments.insert_many(sample_enrollments)
    print(f"Inserted {len(sample_enrollments)} sample enrollment documents.")
except Exception as e:
    print(f"Error inserting sample enrollment documents: {e}")


# --- 4. Sample Lessons Collection Documents ---
sample_lessons = []
lesson_ids = []
lesson_counter = 0

for course_id in course_ids:
    num_lessons_for_course = random.randint(3, 5) # Each course gets 3-5 lessons
    for i in range(num_lessons_for_course):
        lesson_id = str(uuid.uuid4())
        lesson_ids.append(lesson_id)
        lesson_counter += 1
        
        sample_lessons.append({
            "_id": lesson_id,
            "course_id": course_id,
            "title": f"Lesson {i+1}: {random.choice(['Introduction', 'Core Concepts', 'Advanced Topics', 'Practice Session'])}",
            "content": f"Detailed content for lesson {i+1} covering specific topics within the course. This lesson aims to deepen understanding of {random.choice(['algorithms', 'frontend', 'data analysis', 'user experience'])}.",
            "order": i + 1,
            "created_at": datetime.utcnow() - timedelta(days=random.randint(5, 45)),
            "updated_at": datetime.utcnow() - timedelta(days=random.randint(1, 5))
        })
        if lesson_counter >= 25: # Ensure we don't exceed 25 lessons
            break
    if lesson_counter >= 25:
        break


try:
    db.lessons.insert_many(sample_lessons)
    print(f"Inserted {len(sample_lessons)} sample lesson documents.")
except Exception as e:
    print(f"Error inserting sample lesson documents: {e}")

# --- 5. Sample Assignments Collection Documents ---
sample_assignments = []
assignment_ids = []

# Ensure assignments are linked to existing lessons
for i in range(10):
    assignment_id = str(uuid.uuid4())
    assignment_ids.append(assignment_id)
    
    sample_assignments.append({
        "_id": assignment_id,
        "lesson_id": random.choice(lesson_ids), # Link to a random existing lesson
        "title": f"Assignment {i+1}: {random.choice(['Quiz', 'Project', 'Essay', 'Coding Challenge'])}",
        "description": f"Complete this task to demonstrate your understanding of the lesson. It's about {random.choice(['data structures', 'web design', 'marketing strategies', 'scientific principles'])}.",
        "due_date": datetime.utcnow() + timedelta(days=random.randint(7, 21)),
        "max_score": 100,
        "created_at": datetime.utcnow() - timedelta(days=random.randint(3, 10)),
        "updated_at": datetime.utcnow() - timedelta(days=random.randint(0, 3))
    })

try:
    db.assignments.insert_many(sample_assignments)
    print(f"Inserted {len(sample_assignments)} sample assignment documents.")
except Exception as e:
    print(f"Error inserting sample assignment documents: {e}")

# --- 6. Sample Submissions Collection Documents ---
sample_submissions = []

# Ensure submissions are linked to existing assignments and students
for i in range(12):
    submission_id = str(uuid.uuid4())
    
    # Select a random assignment and a random student
    assignment_id = random.choice(assignment_ids)
    student_id = random.choice(student_ids)

    # Randomly assign grade and feedback
    has_grade = random.choice([True, False])
    grade = round(random.uniform(50, 100), 2) if has_grade else None
    feedback = random.choice([
        "Excellent work!", "Good attempt, review chapter 3.",
        "Well done, minor improvements needed.", "Needs more detail.", None
    ]) if has_grade else None

    sample_submissions.append({
        "_id": submission_id,
        "assignment_id": assignment_id,
        "student_id": student_id,
        "submission_date": datetime.utcnow() - timedelta(days=random.randint(0, 7)),
        "content": f"Submission content for assignment {assignment_id[:8]} by student {student_id[:8]}.",
        "grade": grade,
        "feedback": feedback,
        "created_at": datetime.utcnow() - timedelta(days=random.randint(0, 7)),
        "updated_at": datetime.utcnow() - timedelta(days=random.randint(0, 1))
    })

try:
    db.submissions.insert_many(sample_submissions)
    print(f"Inserted {len(sample_submissions)} sample submission documents.")
except Exception as e:
    print(f"Error inserting sample submission documents: {e}")

# Close the connection
client.close()
print("MongoDB connection closed.")

Connected to MongoDB. Inserting sample documents...
Cleared existing data from collections.
Inserted 20 sample user documents.
Inserted 8 sample course documents.
Inserted 15 sample enrollment documents.
Inserted 25 sample lesson documents.
Inserted 10 sample assignment documents.
Inserted 12 sample submission documents.
MongoDB connection closed.


C:\Users\PC\AppData\Local\Temp\ipykernel_16904\118029722.py:53: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow() - timedelta(days=random.randint(30, 365)),
C:\Users\PC\AppData\Local\Temp\ipykernel_16904\118029722.py:54: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "updated_at": datetime.utcnow() - timedelta(days=random.randint(1, 30)),
C:\Users\PC\AppData\Local\Temp\ipykernel_16904\118029722.py:91: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow() - timedelta(d

In [11]:
from pymongo import MongoClient
from datetime import datetime
import uuid
import random

# Establish connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

print("Connected to MongoDB. Performing Create Operations...")

# --- Helper function to check if a document exists by ID ---
def document_exists(collection_name, doc_id):
    """Checks if a document with the given ID exists in the specified collection."""
    return db[collection_name].find_one({"_id": doc_id}) is not None

# --- Operation 1: Add a new student user ---
def add_new_student(username, email, password, first_name, last_name, bio, skills):
    new_student_id = str(uuid.uuid4())
    student_document = {
        "_id": new_student_id,
        "username": username,
        "email": email,
        "password_hash": password, # In a real app, this would be a hashed password
        "role": "student",
        "created_at": datetime.utcnow(),
        "updated_at": datetime.utcnow(),
        "first_name": first_name,
        "last_name": last_name,
        "profile": {
            "bio": bio,
            "avatar": f"https://placehold.co/100x100/{random.choice(['E6E6FA', 'C1E1C1', 'F0F8FF'])}/000000?text={first_name[0]}{last_name[0]}",
            "skills": skills
        },
        "is_active": True
    }
    try:
        db.users.insert_one(student_document)
        print(f"Successfully added new student: {username} (ID: {new_student_id})")
        return new_student_id
    except Exception as e:
        print(f"Error adding new student: {e}")
        return None

# --- Operation 2: Create a new course ---
def create_new_course(title, description, instructor_id, category, level, duration, price, tags):
    # Ensure the instructor exists before creating the course
    if not document_exists("users", instructor_id):
        print(f"Error: Instructor with ID {instructor_id} not found. Cannot create course.")
        return None

    new_course_id = str(uuid.uuid4())
    course_document = {
        "_id": new_course_id,
        "title": title,
        "description": description,
        "instructor_id": instructor_id,
        "category": category,
        "level": level,
        "duration": duration,
        "price": price,
        "tags": tags,
        "created_at": datetime.utcnow(),
        "updated_at": datetime.utcnow(),
        "is_published": True
    }
    try:
        db.courses.insert_one(course_document)
        print(f"Successfully created new course: '{title}' (ID: {new_course_id})")
        return new_course_id
    except Exception as e:
        print(f"Error creating new course: {e}")
        return None

# --- Operation 3: Enroll a student in a course ---
def enroll_student_in_course(student_id, course_id):
    # Ensure student and course exist
    if not document_exists("users", student_id):
        print(f"Error: Student with ID {student_id} not found. Cannot enroll.")
        return None
    if not document_exists("courses", course_id):
        print(f"Error: Course with ID {course_id} not found. Cannot enroll.")
        return None

    # Check if student is already enrolled in this course
    existing_enrollment = db.enrollments.find_one({"student_id": student_id, "course_id": course_id})
    if existing_enrollment:
        print(f"Student {student_id} is already enrolled in course {course_id}.")
        return existing_enrollment["_id"]

    new_enrollment_id = str(uuid.uuid4())
    enrollment_document = {
        "_id": new_enrollment_id,
        "student_id": student_id,
        "course_id": course_id,
        "enrollment_date": datetime.utcnow(),
        "status": "active"
    }
    try:
        db.enrollments.insert_one(enrollment_document)
        print(f"Successfully enrolled student {student_id[:8]} in course {course_id[:8]} (Enrollment ID: {new_enrollment_id})")
        return new_enrollment_id
    except Exception as e:
        print(f"Error enrolling student: {e}")
        return None

# --- Operation 4: Add a new lesson to an existing course ---
def add_new_lesson(course_id, title, content):
    # Ensure the course exists
    if not document_exists("courses", course_id):
        print(f"Error: Course with ID {course_id} not found. Cannot add lesson.")
        return None

    # Determine the next order number for the lesson in this course
    # Find the maximum order number for lessons in this course
    last_lesson = db.lessons.find({"course_id": course_id}).sort("order", -1).limit(1)
    last_lesson_doc = next(last_lesson, None) # Get the first (and only) document
    next_order = (last_lesson_doc['order'] + 1) if last_lesson_doc else 1

    new_lesson_id = str(uuid.uuid4())
    lesson_document = {
        "_id": new_lesson_id,
        "course_id": course_id,
        "title": title,
        "content": content,
        "order": next_order,
        "created_at": datetime.utcnow(),
        "updated_at": datetime.utcnow()
    }
    try:
        db.lessons.insert_one(lesson_document)
        print(f"Successfully added new lesson: '{title}' to course {course_id[:8]} (Lesson ID: {new_lesson_id})")
        return new_lesson_id
    except Exception as e:
        print(f"Error adding new lesson: {e}")
        return None

# --- Example Usage of the functions ---
if __name__ == "__main__":
    print("\n--- Running Create Operations Examples ---")

    # Example 1: Add a new student user
    new_student_user_id = add_new_student(
        "new_student_galore",
        "galore.jones@example.com",
        "secure_password123",
        "Galore",
        "Jones",
        "New student eager to learn Python and data science.",
        ["Python", "Beginner"]
    )

    # Fetch an existing instructor ID from the database for creating a new course
    # You could also use a specific instructor_id you know from your sample data
    existing_instructor = db.users.find_one({"role": "instructor"})
    if existing_instructor:
        instructor_id_for_new_course = existing_instructor["_id"]
        print(f"\nUsing existing instructor ID: {instructor_id_for_new_course[:8]} for new course.")
    else:
        print("\nNo existing instructor found. Please run the sample data script first or add an instructor.")
        instructor_id_for_new_course = None # Prevent errors if no instructor

    # Example 2: Create a new course
    new_course_id = None
    if instructor_id_for_new_course:
        new_course_id = create_new_course(
            "Introduction to AI Ethics",
            "This course explores the ethical implications of artificial intelligence and its impact on society.",
            instructor_id_for_new_course,
            "AI & Ethics",
            "intermediate",
            30,
            99.99,
            ["AI", "Ethics", "Philosophy", "Technology"]
        )

    # Example 3: Enroll the new student in the new course (if both were created)
    if new_student_user_id and new_course_id:
        enrollment_id = enroll_student_in_course(new_student_user_id, new_course_id)
    else:
        print("\nSkipping enrollment: New student or new course not created.")

    # Example 4: Add a new lesson to an existing course
    # We'll try to add a lesson to the 'Introduction to AI Ethics' course if it was created
    if new_course_id:
        add_new_lesson(
            new_course_id,
            "Lesson 1: Foundations of AI Ethics",
            "This lesson introduces the fundamental concepts and historical context of AI ethics, including key frameworks and principles."
        )
        add_new_lesson( # Add another lesson to demonstrate order increment
            new_course_id,
            "Lesson 2: Bias and Fairness in AI",
            "Explore issues of bias in AI systems, methods for detection, and approaches to promoting fairness in algorithms and data."
        )
    else:
        print("\nSkipping adding lesson: New course not created.")

    # Close the connection
    client.close()
    print("\nMongoDB connection closed.")

Connected to MongoDB. Performing Create Operations...

--- Running Create Operations Examples ---
Successfully added new student: new_student_galore (ID: 083a14ed-c3ed-4948-a62d-00a66764559d)

Using existing instructor ID: 8c41e90c for new course.
Successfully created new course: 'Introduction to AI Ethics' (ID: 20f458a6-72e1-4281-837e-2db3db73a8da)
Successfully enrolled student 083a14ed in course 20f458a6 (Enrollment ID: 46de6906-e598-4e99-8e10-77256d148134)
Successfully added new lesson: 'Lesson 1: Foundations of AI Ethics' to course 20f458a6 (Lesson ID: b21c6b70-f3e3-4479-b05e-f16d8cadaa59)
Successfully added new lesson: 'Lesson 2: Bias and Fairness in AI' to course 20f458a6 (Lesson ID: bfc69df6-7627-4e95-8662-81e7c7f975df)

MongoDB connection closed.


C:\Users\PC\AppData\Local\Temp\ipykernel_16904\2483902847.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow(),
C:\Users\PC\AppData\Local\Temp\ipykernel_16904\2483902847.py:27: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "updated_at": datetime.utcnow(),
C:\Users\PC\AppData\Local\Temp\ipykernel_16904\2483902847.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow(),
C:\Users\PC\AppData\Local\Temp\ipykernel_16904\2483902847.py:64: DeprecationWarning: dateti

In [17]:
from pymongo import MongoClient
from bson.regex import Regex # For case-insensitive partial matching
import re # Import the re module for regex escaping
from datetime import datetime # Import datetime for date formatting in print_documents

# Establish connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

print("Connected to MongoDB. Performing Read Operations...")

# --- Helper function to display documents cleanly ---
def print_documents(title, documents):
    print(f"\n--- {title} ---")
    if not documents:
        print("No documents found.")
        return
    for doc in documents:
        # Create a copy to avoid modifying the original doc during printing
        doc_copy = doc.copy()
        # Format datetimes for better readability
        for key, value in doc_copy.items():
            if isinstance(value, datetime):
                doc_copy[key] = value.strftime("%Y-%m-%d %H:%M:%S UTC")
            # Truncate long strings for display purposes, especially IDs
            if isinstance(value, str) and len(value) > 20 and '_id' not in key.lower():
                doc_copy[key] = value[:17] + "..."
            elif isinstance(value, str) and len(value) > 10 and 'id' in key.lower() and key != '_id':
                doc_copy[key] = value[:7] + "..."
        print(doc_copy)
    print("-" * (len(title) + 8))

# --- Operation 1: Find all active students ---
def find_active_students():
    """Retrieves all users with the role 'student' and 'is_active' status as True."""
    print("\nAttempting to find all active students...")
    active_students = db.users.find({"role": "student", "is_active": True})
    return list(active_students) # Convert cursor to list for printing

# --- Operation 2: Retrieve course details with instructor information ---
def get_course_details_with_instructor(course_title_substring=None):
    """
    Retrieves course details and "joins" them with their instructor's first name, last name, and email.
    Can filter by a substring of the course title.
    """
    print(f"\nAttempting to retrieve course details with instructor info (matching '{course_title_substring}' if provided)...")
    
    match_stage = {}
    if course_title_substring:
        # Using re.escape for the substring to ensure any regex special characters are treated literally
        # Then, constructing the regex pattern for partial (contains) and case-insensitive matching
        escaped_substring = re.escape(course_title_substring)
        match_stage = {"title": {"$regex": escaped_substring, "$options": "i"}} 

    pipeline = [
        {"$match": match_stage}, # Filter courses if a title substring is provided
        {
            "$lookup": {
                "from": "users",          # The collection to join with
                "localField": "instructor_id", # Field from the input documents (courses)
                "foreignField": "_id",    # Field from the "from" documents (users)
                "as": "instructor_info"   # Output array field name
            }
        },
        {"$unwind": "$instructor_info"}, # Deconstructs the array field from the $lookup operation
        {
            "$project": { # Select and rename fields for clarity
                "_id": 1,
                "title": 1,
                "description": 1,
                "category": 1,
                "level": 1,
                "price": 1,
                "instructor_name": {"$concat": ["$instructor_info.first_name", " ", "$instructor_info.last_name"]},
                "instructor_email": "$instructor_info.email"
            }
        }
    ]
    
    courses_with_instructor = db.courses.aggregate(pipeline)
    return list(courses_with_instructor)

# --- Operation 3: Get all courses in a specific category ---
def get_courses_by_category(category_name):
    """Retrieves all courses belonging to a specific category."""
    print(f"\nAttempting to retrieve courses in category: '{category_name}'...")
    courses = db.courses.find({"category": category_name})
    return list(courses)

# --- Operation 4: Find students enrolled in a particular course ---
def find_students_in_course(course_title):
    """
    Finds all students enrolled in a course identified by its title.
    Performs a lookup on enrollments and then on users.
    """
    print(f"\nAttempting to find students enrolled in course: '{course_title}'...")
    
    # First, find the course_id for the given title (case-insensitive)
    # Use re.escape to make sure any special regex characters in course_title are treated literally
    escaped_course_title = re.escape(course_title)
    course_doc = db.courses.find_one({"title": {"$regex": f"^{escaped_course_title}$", "$options": "i"}})
    
    if not course_doc:
        print(f"Error: Course '{course_title}' not found.")
        return []
    
    course_id = course_doc["_id"]
    print(f"Found course ID: {course_id[:8]} for '{course_title}'.")

    pipeline = [
        {"$match": {"course_id": course_id}}, # Match enrollments for the specific course
        {
            "$lookup": {
                "from": "users",          # Join with users collection
                "localField": "student_id", # Field from enrollments
                "foreignField": "_id",    # Field from users
                "as": "student_info"      # Output array
            }
        },
        {"$unwind": "$student_info"}, # Deconstruct the student_info array
        {
            "$project": { # Select relevant student details
                "_id": "$student_info._id", # The student's ID
                "username": "$student_info.username",
                "email": "$student_info.email",
                "first_name": "$student_info.first_name",
                "last_name": "$student_info.last_name",
                "enrollment_date": "$enrollment_date",
                "enrollment_status": "$status"
            }
        }
    ]
    
    enrolled_students = db.enrollments.aggregate(pipeline)
    return list(enrolled_students)

# --- Operation 5: Search courses by title (case-insensitive, partial match) ---
def search_courses_by_title_partial(search_term):
    """
    Searches for courses where the title contains the given search term,
    case-insensitively.
    """
    print(f"\nAttempting to search courses by title containing: '{search_term}'...")
    # Using re.escape for the search_term to ensure any regex special characters are treated literally
    escaped_search_term = re.escape(search_term)
    # Using $regex for partial and case-insensitive matching
    # $options: "i" makes it case-insensitive
    courses = db.courses.find({"title": {"$regex": escaped_search_term, "$options": "i"}})
    return list(courses)

# --- Example Usage of the functions ---
if __name__ == "__main__":
    # Ensure there's some data in the database by running the sample_data script first
    # Or rely on data inserted by the create_operations script if run previously

    # Example 1: Find all active students
    active_students_docs = find_active_students()
    print_documents("Active Students", active_students_docs)

    # Example 2: Retrieve course details with instructor information
    # Get all courses with instructor info
    all_courses_with_instructors = get_course_details_with_instructor()
    print_documents("All Course Details with Instructor Info", all_courses_with_instructors)

    # Get a specific course with instructor info
    ai_ethics_course_with_instructor = get_course_details_with_instructor("AI Ethics")
    print_documents("AI Ethics Course Details with Instructor Info", ai_ethics_course_with_instructor)


    # Example 3: Get all courses in a specific category
    programming_courses = get_courses_by_category("Programming")
    print_documents("Courses in 'Programming' Category", programming_courses)

    # Example 4: Find students enrolled in a particular course
    students_in_ai_ethics = find_students_in_course("Introduction to AI Ethics")
    print_documents("Students Enrolled in 'Introduction to AI Ethics'", students_in_ai_ethics)

    students_in_python_basics = find_students_in_course("Introduction to Python Programming")
    print_documents("Students Enrolled in 'Introduction to Python Programming'", students_in_python_basics)

    # Example 5: Search courses by title (case-insensitive, partial match)
    search_term = "python"
    python_courses_search = search_courses_by_title_partial(search_term)
    print_documents(f"Courses matching title '{search_term}'", python_courses_search)

    search_term_web = "web"
    web_courses_search = search_courses_by_title_partial(search_term_web)
    print_documents(f"Courses matching title '{search_term_web}'", web_courses_search)


    # Close the connection
    client.close()
    print("\nMongoDB connection closed.")

Connected to MongoDB. Performing Read Operations...

Attempting to find all active students...

--- Active Students ---
{'_id': '09fedbfd-ff5f-48ff-839a-058d0f0a7d52', 'username': 'ivan_thomas2', 'email': 'ivan.thomas2@exam...', 'password_hash': 'hashed_password_5...', 'role': 'student', 'created_at': '2025-03-05 15:46:34 UTC', 'updated_at': '2025-05-17 15:46:34 UTC', 'first_name': 'Ivan', 'last_name': 'Thomas', 'profile': {'bio': 'Eager to learn about design.', 'avatar': 'https://placehold.co/100x100/FFDDC1/000000?text=IT', 'skills': ['Fast Learner', 'Beginner']}, 'is_active': True}
{'_id': 'f6c049c4-61c2-4add-aeec-932f995aec3c', 'username': 'karl_martin4', 'email': 'karl.martin4@exam...', 'password_hash': 'hashed_password_f...', 'role': 'student', 'created_at': '2024-06-18 15:46:34 UTC', 'updated_at': '2025-06-07 15:46:34 UTC', 'first_name': 'Karl', 'last_name': 'Martin', 'profile': {'bio': 'Eager to learn about programming.', 'avatar': 'https://placehold.co/100x100/ADD8E6/000000?tex

In [19]:
from pymongo import MongoClient
from datetime import datetime
import uuid # For generating temporary IDs if needed for examples
import random # For random choices in examples

# Establish connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

print("Connected to MongoDB. Performing Update Operations...")

# --- Helper function to display documents cleanly ---
def print_documents(title, documents):
    print(f"\n--- {title} ---")
    if not documents:
        print("No documents found.")
        return
    for doc in documents:
        # Create a copy to avoid modifying the original doc during printing
        doc_copy = doc.copy()
        # Format datetimes for better readability
        for key, value in doc_copy.items():
            if isinstance(value, datetime):
                doc_copy[key] = value.strftime("%Y-%m-%d %H:%M:%S UTC")
            # Truncate long strings for display purposes, especially IDs
            if isinstance(value, str) and len(value) > 20 and '_id' not in key.lower():
                doc_copy[key] = value[:17] + "..."
            elif isinstance(value, str) and len(value) > 10 and 'id' in key.lower() and key != '_id':
                doc_copy[key] = value[:7] + "..."
        print(doc_copy)
    print("-" * (len(title) + 8))

# --- Operation 1: Update a user’s profile information ---
def update_user_profile(user_id, new_bio=None, new_avatar=None, skills_to_add=None, is_active=None):
    """
    Updates a user's profile information.
    :param user_id: The _id of the user to update.
    :param new_bio: Optional new biography string.
    :param new_avatar: Optional new avatar URL.
    :param skills_to_add: A list of new skills to add to the existing skills array.
    :param is_active: Optional boolean to change active status.
    """
    update_fields = {}
    if new_bio is not None:
        update_fields["profile.bio"] = new_bio
    if new_avatar is not None:
        update_fields["profile.avatar"] = new_avatar
    if is_active is not None:
        update_fields["is_active"] = is_active

    # Add skills using $addToSet to avoid duplicates in the array
    if skills_to_add:
        update_fields["profile.skills"] = {"$each": skills_to_add} # Add multiple items to array
        
    update_fields["updated_at"] = datetime.utcnow() # Always update the 'updated_at' timestamp

    try:
        if skills_to_add:
            # For $addToSet with $each, we need to handle it separately or use update_one with multiple operators
            # PyMongo allows multiple operators in one update call.
            result = db.users.update_one(
                {"_id": user_id},
                {
                    "$set": {k: v for k, v in update_fields.items() if k != "profile.skills"},
                    "$addToSet": {"profile.skills": {"$each": skills_to_add}} if skills_to_add else {}
                }
            )
        else:
            result = db.users.update_one(
                {"_id": user_id},
                {"$set": update_fields}
            )


        if result.matched_count > 0:
            print(f"Successfully updated user {user_id[:8]} profile. Matched: {result.matched_count}, Modified: {result.modified_count}")
        else:
            print(f"User {user_id[:8]} not found or no changes made.")
    except Exception as e:
        print(f"Error updating user profile: {e}")

# --- Operation 2: Mark a course as published ---
def mark_course_as_published(course_id, is_published=True):
    """
    Marks a course as published (or unpublished).
    :param course_id: The _id of the course to update.
    :param is_published: Boolean indicating the published status.
    """
    try:
        result = db.courses.update_one(
            {"_id": course_id},
            {"$set": {"is_published": is_published, "updated_at": datetime.utcnow()}}
        )
        if result.matched_count > 0:
            status = "published" if is_published else "unpublished"
            print(f"Successfully marked course {course_id[:8]} as {status}. Matched: {result.matched_count}, Modified: {result.modified_count}")
        else:
            print(f"Course {course_id[:8]} not found or no changes made.")
    except Exception as e:
        print(f"Error marking course as published: {e}")

# --- Operation 3: Update assignment grades ---
def update_assignment_grade(submission_id, new_grade, feedback=None):
    """
    Updates the grade and optionally feedback for an assignment submission.
    :param submission_id: The _id of the submission to update.
    :param new_grade: The new numeric grade (0-100).
    :param feedback: Optional feedback string.
    """
    if not (0 <= new_grade <= 100):
        print(f"Error: Grade {new_grade} is out of valid range (0-100).")
        return

    update_fields = {
        "grade": float(new_grade), # Ensure it's a float for BSON double type
        "updated_at": datetime.utcnow()
    }
    if feedback is not None:
        update_fields["feedback"] = feedback
    
    try:
        result = db.submissions.update_one(
            {"_id": submission_id},
            {"$set": update_fields}
        )
        if result.matched_count > 0:
            print(f"Successfully updated grade for submission {submission_id[:8]} to {new_grade}. Matched: {result.matched_count}, Modified: {result.modified_count}")
        else:
            print(f"Submission {submission_id[:8]} not found or no changes made.")
    except Exception as e:
        print(f"Error updating assignment grade: {e}")

# --- Operation 4: Add tags to an existing course ---
def add_tags_to_course(course_id, new_tags):
    """
    Adds new tags to the tags array of an existing course.
    Uses $addToSet to prevent duplicate tags.
    :param course_id: The _id of the course to update.
    :param new_tags: A list of tags to add.
    """
    try:
        result = db.courses.update_one(
            {"_id": course_id},
            {"$addToSet": {"tags": {"$each": new_tags}}, # Add multiple tags to array, avoid duplicates
             "$set": {"updated_at": datetime.utcnow()}} # Update timestamp
        )
        if result.matched_count > 0:
            print(f"Successfully added tags to course {course_id[:8]}. Matched: {result.matched_count}, Modified: {result.modified_count}")
        else:
            print(f"Course {course_id[:8]} not found or no changes made.")
    except Exception as e:
        print(f"Error adding tags to course: {e}")


# --- Example Usage of the functions ---
if __name__ == "__main__":
    print("\n--- Running Update Operations Examples ---")

    # Fetch existing IDs for examples
    first_student = db.users.find_one({"role": "student"})
    first_course = db.courses.find_one({}) # Get any course
    
    # Try to find a submission that hasn't been graded yet (grade is None)
    # This assumes the sample data or create operations have been run.
    ungraded_submission = db.submissions.find_one({"grade": None})
    
    # --- Example 1: Update a user’s profile information ---
    if first_student:
        print(f"\nUpdating profile for student: {first_student['username']} (ID: {first_student['_id'][:8]})")
        update_user_profile(
            first_student["_id"],
            new_bio="Enthusiastic learner focusing on advanced topics.",
            skills_to_add=["Machine Learning", "Cloud Computing"],
            is_active=True
        )
        # Verify update
        updated_student = db.users.find_one({"_id": first_student["_id"]})
        print_documents("Updated Student Profile", [updated_student])
    else:
        print("\nNo student found for profile update example. Please run sample data script.")

    # --- Example 2: Mark a course as published ---
    if first_course:
        print(f"\nMarking course: '{first_course['title']}' (ID: {first_course['_id'][:8]}) as published...")
        mark_course_as_published(first_course["_id"], True)
        # Verify update
        updated_course = db.courses.find_one({"_id": first_course["_id"]})
        print_documents("Updated Course Published Status", [updated_course])
    else:
        print("\nNo course found for publish status update. Please run sample data script.")
    
    # --- Example 3: Update assignment grades ---
    if ungraded_submission:
        print(f"\nUpdating grade for submission ID: {ungraded_submission['_id'][:8]}")
        update_assignment_grade(ungraded_submission["_id"], 92.5, "Excellent work! Very clear and concise.")
        # Verify update
        updated_submission = db.submissions.find_one({"_id": ungraded_submission["_id"]})
        print_documents("Updated Submission Grade", [updated_submission])
    else:
        print("\nNo ungraded submission found for grade update example. You might need to run sample data or create operations again.")
        # As a fallback, try to update a random graded submission if no ungraded ones
        graded_submission = db.submissions.find_one({"grade": {"$ne": None}})
        if graded_submission:
            print(f"\nUpdating grade for a random graded submission ID: {graded_submission['_id'][:8]}")
            update_assignment_grade(graded_submission["_id"], 99.0, "Revised and improved, outstanding!")
            updated_submission = db.submissions.find_one({"_id": graded_submission["_id"]})
            print_documents("Updated Graded Submission (Fallback)", [updated_submission])


    # --- Example 4: Add tags to an existing course ---
    if first_course:
        print(f"\nAdding tags to course: '{first_course['title']}' (ID: {first_course['_id'][:8]})")
        add_tags_to_course(first_course["_id"], ["NewTag1", "Advanced", "Interactive"])
        # Verify update
        updated_course_tags = db.courses.find_one({"_id": first_course["_id"]})
        print_documents("Course with Added Tags", [updated_course_tags])
    else:
        print("\nNo course found for adding tags example. Please run sample data script.")

    # Close the connection
    client.close()
    print("\nMongoDB connection closed.")

Connected to MongoDB. Performing Update Operations...

--- Running Update Operations Examples ---

Updating profile for student: grace_anderson0 (ID: 32c92c75)
Successfully updated user 32c92c75 profile. Matched: 1, Modified: 1

--- Updated Student Profile ---
{'_id': '32c92c75-cd8c-4661-9777-56905eb7431a', 'username': 'grace_anderson0', 'email': 'grace.anderson0@e...', 'password_hash': 'hashed_password_9...', 'role': 'student', 'created_at': '2025-01-25 15:46:34 UTC', 'updated_at': '2025-06-14 16:41:09 UTC', 'first_name': 'Grace', 'last_name': 'Anderson', 'profile': {'bio': 'Enthusiastic learner focusing on advanced topics.', 'avatar': 'https://placehold.co/100x100/ADD8E6/000000?text=GA', 'skills': ['Fast Learner', 'Machine Learning', 'Cloud Computing']}, 'is_active': True}
-------------------------------

Marking course: 'Mastering Science Course 1' (ID: e75448f9) as published...
Successfully marked course e75448f9 as published. Matched: 1, Modified: 1

--- Updated Course Published S

C:\Users\PC\AppData\Local\Temp\ipykernel_16904\3827043335.py:55: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  update_fields["updated_at"] = datetime.utcnow() # Always update the 'updated_at' timestamp
C:\Users\PC\AppData\Local\Temp\ipykernel_16904\3827043335.py:92: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  {"$set": {"is_published": is_published, "updated_at": datetime.utcnow()}}
C:\Users\PC\AppData\Local\Temp\ipykernel_16904\3827043335.py:116: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "updated_at": datetime.u

In [23]:
from pymongo import MongoClient
from datetime import datetime, UTC # Import UTC from datetime
import uuid # For generating temporary IDs if needed for examples

# Establish connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

print("Connected to MongoDB. Performing Delete Operations...")

# --- Helper function to display documents cleanly ---
def print_documents(title, documents):
    print(f"\n--- {title} ---")
    if not documents:
        print("No documents found.")
        return
    for doc in documents:
        if doc is None: # Added check for None document
            print("  [Document not found or deleted]")
            continue

        # Create a copy to avoid modifying the original doc during printing
        doc_copy = doc.copy()
        # Format datetimes for better readability
        for key, value in doc_copy.items():
            if isinstance(value, datetime):
                # Ensure datetime objects are converted to UTC for consistency before formatting
                if value.tzinfo is None: # If naive datetime, assume UTC for display purposes
                    doc_copy[key] = value.replace(tzinfo=UTC).strftime("%Y-%m-%d %H:%M:%S UTC")
                else:
                    doc_copy[key] = value.astimezone(UTC).strftime("%Y-%m-%d %H:%M:%S UTC")
            # Truncate long strings for display purposes, especially IDs
            if isinstance(value, str) and len(value) > 20 and '_id' not in key.lower():
                doc_copy[key] = value[:17] + "..."
            elif isinstance(value, str) and len(value) > 10 and 'id' in key.lower() and key != '_id':
                doc_copy[key] = value[:7] + "..."
        print(doc_copy)
    print("-" * (len(title) + 8))

# --- Operation 1: Remove a user (soft delete by setting isActive to false) ---
def soft_delete_user(user_id):
    """
    Performs a soft delete on a user by setting their 'is_active' status to False.
    This is generally preferred over hard deleting users to retain historical data.
    """
    try:
        result = db.users.update_one(
            {"_id": user_id},
            {"$set": {"is_active": False, "updated_at": datetime.now(UTC)}} # Updated to datetime.now(UTC)
        )
        if result.matched_count > 0:
            print(f"Successfully soft-deleted user {user_id[:8]} (is_active set to False).")
        else:
            print(f"User {user_id[:8]} not found or already inactive.")
    except Exception as e:
        print(f"Error soft-deleting user: {e}")

# --- Operation 2: Delete an enrollment ---
def delete_enrollment(enrollment_id):
    """
    Permanently deletes an enrollment document from the 'enrollments' collection.
    """
    try:
        result = db.enrollments.delete_one({"_id": enrollment_id})
        if result.deleted_count > 0:
            print(f"Successfully deleted enrollment {enrollment_id[:8]}.")
        else:
            print(f"Enrollment {enrollment_id[:8]} not found.")
    except Exception as e:
        print(f"Error deleting enrollment: {e}")

# --- Operation 3: Remove a lesson from a course ---
def remove_lesson_from_course(lesson_id):
    """
    Removes a specific lesson from the 'lessons' collection and
    then re-orders the remaining lessons in the same course.
    """
    try:
        # 1. Find the lesson to get its course_id and current order
        lesson_to_delete = db.lessons.find_one({"_id": lesson_id})
        if not lesson_to_delete:
            print(f"Lesson {lesson_id[:8]} not found. Cannot remove.")
            return

        course_id = lesson_to_delete["course_id"]
        deleted_order = lesson_to_delete["order"]

        # 2. Delete the lesson
        delete_result = db.lessons.delete_one({"_id": lesson_id})
        if delete_result.deleted_count > 0:
            print(f"Successfully deleted lesson {lesson_id[:8]} from course {course_id[:8]}.")

            # 3. Re-order remaining lessons in the same course
            # Find all lessons in the same course with an order greater than the deleted lesson's order
            lessons_to_reorder = db.lessons.find(
                {"course_id": course_id, "order": {"$gt": deleted_order}}
            ).sort("order", 1) # Sort by order to process sequentially

            updates_performed = 0
            for lesson in lessons_to_reorder:
                # Decrement the order number
                new_order = lesson["order"] - 1
                update_result = db.lessons.update_one(
                    {"_id": lesson["_id"]},
                    {"$set": {"order": new_order, "updated_at": datetime.now(UTC)}} # Updated to datetime.now(UTC)
                )
                if update_result.modified_count > 0:
                    updates_performed += 1
            
            if updates_performed > 0:
                print(f"Re-ordered {updates_performed} subsequent lessons in course {course_id[:8]}.")
            else:
                print(f"No lessons needed re-ordering after deleting lesson {lesson_id[:8]}.")
        else:
            print(f"Lesson {lesson_id[:8]} not found. No deletion performed.")

    except Exception as e:
        print(f"Error removing lesson: {e}")

# --- Example Usage of the functions ---
if __name__ == "__main__":
    print("\n--- Running Delete Operations Examples ---")

    # Fetch existing IDs for examples
    # Soft Delete User Example
    user_to_soft_delete = db.users.find_one({"role": "student", "is_active": True})
    if user_to_soft_delete:
        print(f"\nAttempting to soft-delete user: {user_to_soft_delete['username']} (ID: {user_to_soft_delete['_id'][:8]})")
        soft_delete_user(user_to_soft_delete["_id"])
        updated_user = db.users.find_one({"_id": user_to_soft_delete["_id"]})
        print_documents("User after Soft Delete", [updated_user])
    else:
        print("\nNo active student found for soft-delete example. Please run sample data script.")

    # Delete Enrollment Example
    enrollment_to_delete = db.enrollments.find_one({}) # Get any enrollment
    if enrollment_to_delete:
        print(f"\nAttempting to delete enrollment: (ID: {enrollment_to_delete['_id'][:8]})")
        print_documents("Enrollment before Deletion", [enrollment_to_delete])
        delete_enrollment(enrollment_to_delete["_id"])
        # Verify deletion - this will correctly return None if deleted
        deleted_check = db.enrollments.find_one({"_id": enrollment_to_delete["_id"]})
        print_documents("Enrollment after Deletion Check", [deleted_check]) # This will now show "[Document not found or deleted]"
    else:
        print("\nNo enrollment found for deletion example. Please run sample data script.")

    # Remove Lesson Example
    # Find a lesson that is not the last one in its course to demonstrate re-ordering
    lesson_to_remove = None
    lessons_in_course = {} # Group lessons by course_id
    for lesson in db.lessons.find({}).sort([("course_id", 1), ("order", 1)]):
        lessons_in_course.setdefault(lesson["course_id"], []).append(lesson)

    # Find a course with at least 2 lessons and pick the first one (not the last)
    for course_id, lessons in lessons_in_course.items():
        if len(lessons) > 1:
            lesson_to_remove = lessons[0] # Pick the first lesson to remove
            break
            
    if lesson_to_remove:
        course_id_of_lesson = lesson_to_remove["course_id"]
        print(f"\nAttempting to remove lesson: '{lesson_to_remove['title']}' (ID: {lesson_to_remove['_id'][:8]}) from course {course_id_of_lesson[:8]}")
        
        print_documents(f"Lessons in course {course_id_of_lesson[:8]} before removal", 
                       list(db.lessons.find({"course_id": course_id_of_lesson}).sort("order", 1)))
        
        remove_lesson_from_course(lesson_to_remove["_id"])
        
        print_documents(f"Lessons in course {course_id_of_lesson[:8]} after removal", 
                       list(db.lessons.find({"course_id": course_id_of_lesson}).sort("order", 1)))
    else:
        print("\nNo suitable lesson found for removal example (need a course with at least 2 lessons). Please run sample data script.")

    # Close the connection
    client.close()
    print("\nMongoDB connection closed.")

Connected to MongoDB. Performing Delete Operations...

--- Running Delete Operations Examples ---

Attempting to soft-delete user: ivan_thomas2 (ID: 09fedbfd)
Successfully soft-deleted user 09fedbfd (is_active set to False).

--- User after Soft Delete ---
{'_id': '09fedbfd-ff5f-48ff-839a-058d0f0a7d52', 'username': 'ivan_thomas2', 'email': 'ivan.thomas2@exam...', 'password_hash': 'hashed_password_5...', 'role': 'student', 'created_at': '2025-03-05 15:46:34 UTC', 'updated_at': '2025-06-14 16:47:02 UTC', 'first_name': 'Ivan', 'last_name': 'Thomas', 'profile': {'bio': 'Eager to learn about design.', 'avatar': 'https://placehold.co/100x100/FFDDC1/000000?text=IT', 'skills': ['Fast Learner', 'Beginner']}, 'is_active': False}
------------------------------

Attempting to delete enrollment: (ID: f9126b7e)

--- Enrollment before Deletion ---
{'_id': 'f9126b7e-76b2-4e57-bd54-a14fca1e2f0a', 'student_id': 'f6c049c...', 'course_id': 'e75448f...', 'enrollment_date': '2025-04-24 15:46:34 UTC', 'statu

In [25]:
from pymongo import MongoClient
from datetime import datetime, timedelta, UTC # Import UTC for timezone-aware datetimes
import re # For regex operations

# Establish connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

print("Connected to MongoDB. Performing Advanced Queries...")

# --- Helper function to display documents cleanly ---
def print_documents(title, documents):
    print(f"\n--- {title} ---")
    if not documents:
        print("No documents found.")
        return
    for doc in documents:
        if doc is None:
            print("  [Document not found or deleted]")
            continue

        # Create a copy to avoid modifying the original doc during printing
        doc_copy = doc.copy()
        # Format datetimes for better readability
        for key, value in doc_copy.items():
            if isinstance(value, datetime):
                if value.tzinfo is None: # If naive datetime, assume UTC for display purposes
                    doc_copy[key] = value.replace(tzinfo=UTC).strftime("%Y-%m-%d %H:%M:%S UTC")
                else:
                    doc_copy[key] = value.astimezone(UTC).strftime("%Y-%m-%d %H:%M:%S UTC")
            # Truncate long strings for display purposes, especially IDs
            if isinstance(value, str) and len(value) > 20 and '_id' not in key.lower():
                doc_copy[key] = value[:17] + "..."
            elif isinstance(value, str) and len(value) > 10 and 'id' in key.lower() and key != '_id':
                doc_copy[key] = value[:7] + "..."
        print(doc_copy)
    print("-" * (len(title) + 8))

# --- Query 1: Find courses with price between $50 and $200 ---
def find_courses_by_price_range(min_price, max_price):
    """
    Finds courses whose price is within a specified range (inclusive).
    Uses $gte (greater than or equal) and $lte (less than or equal) operators.
    """
    print(f"\nAttempting to find courses with price between ${min_price} and ${max_price}...")
    courses = db.courses.find({
        "price": {
            "$gte": min_price,
            "$lte": max_price
        }
    })
    return list(courses)

# --- Query 2: Get users who joined in the last 6 months ---
def get_users_joined_last_n_months(num_months):
    """
    Retrieves users whose 'created_at' date is within the last 'num_months'.
    Uses $gte operator with a calculated datetime.
    """
    print(f"\nAttempting to find users who joined in the last {num_months} months...")
    
    # Calculate the date 'num_months' ago from now (UTC)
    six_months_ago = datetime.now(UTC) - timedelta(days=num_months * 30) # Approx 30 days per month
    
    users = db.users.find({
        "created_at": {
            "$gte": six_months_ago
        }
    })
    return list(users)

# --- Query 3: Find courses that have specific tags using $in operator ---
def find_courses_with_specific_tags(tags_list):
    """
    Finds courses that have at least one of the specified tags in their 'tags' array.
    Uses the $in operator for array matching.
    """
    print(f"\nAttempting to find courses with any of these tags: {tags_list}...")
    courses = db.courses.find({
        "tags": {
            "$in": tags_list
        }
    })
    return list(courses)

# --- Query 4: Retrieve assignments with due dates in the next week ---
def get_assignments_due_next_week():
    """
    Retrieves assignments whose 'due_date' falls within the next 7 days from now (UTC).
    Uses $gte and $lte operators.
    """
    print("\nAttempting to retrieve assignments due in the next week...")
    
    now_utc = datetime.now(UTC)
    one_week_from_now_utc = now_utc + timedelta(days=7)
    
    assignments = db.assignments.find({
        "due_date": {
            "$gte": now_utc,
            "$lte": one_week_from_now_utc
        }
    })
    return list(assignments)


# --- Example Usage of the functions ---
if __name__ == "__main__":
    print("\n--- Running Advanced Queries Examples ---")

    # Example 1: Find courses with price between $50 and $200
    courses_50_to_200 = find_courses_by_price_range(50, 200)
    print_documents("Courses with Price between $50 and $200", courses_50_to_200)

    # Example 2: Get users who joined in the last 6 months
    users_joined_recent = get_users_joined_last_n_months(6)
    print_documents("Users Joined in the Last 6 Months", users_joined_recent)
    
    # Example 3: Find courses that have specific tags using $in operator
    specific_tags = ["Python", "Web Development", "Interactive"]
    courses_with_tags = find_courses_with_specific_tags(specific_tags)
    print_documents(f"Courses with tags: {specific_tags}", courses_with_tags)

    # Example 4: Retrieve assignments with due dates in the next week
    assignments_next_week = get_assignments_due_next_week()
    print_documents("Assignments Due in the Next Week", assignments_next_week)

    # Close the connection
    client.close()
    print("\nMongoDB connection closed.")

Connected to MongoDB. Performing Advanced Queries...

--- Running Advanced Queries Examples ---

Attempting to find courses with price between $50 and $200...

--- Courses with Price between $50 and $200 ---
{'_id': 'e75448f9-dc22-4e1b-a1f7-99b613544b1e', 'title': 'Mastering Science...', 'description': 'A comprehensive c...', 'instructor_id': '8c41e90...', 'category': 'Programming', 'level': 'beginner', 'duration': 31, 'price': 156.87, 'tags': ['Certification', 'Self-paced', 'Project-based', 'Interactive', 'NewTag1', 'Advanced'], 'created_at': '2025-01-16 15:46:34 UTC', 'updated_at': '2025-06-14 16:41:09 UTC', 'is_published': True}
{'_id': 'cb56f318-691f-4e4e-bdc5-cd59de226a7e', 'title': 'Introduction to A...', 'description': 'A comprehensive c...', 'instructor_id': '22789ea...', 'category': 'Programming', 'level': 'beginner', 'duration': 74, 'price': 110.69, 'tags': ['Interactive', 'Beginner Friendly', 'Online'], 'created_at': '2024-12-26 15:46:34 UTC', 'updated_at': '2025-05-01 15:46

In [27]:
from pymongo import MongoClient
from datetime import datetime, timedelta, UTC
import re

# Establish connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

print("Connected to MongoDB. Performing Advanced Aggregation Pipelines...")

# --- Helper function to display documents cleanly ---
def print_documents(title, documents):
    print(f"\n--- {title} ---")
    if not documents:
        print("No documents found.")
        return
    for doc in documents:
        if doc is None:
            print("  [Document not found or deleted]")
            continue

        # Create a copy to avoid modifying the original doc during printing
        doc_copy = doc.copy()
        # Format datetimes for better readability
        for key, value in doc_copy.items():
            if isinstance(value, datetime):
                if value.tzinfo is None: # If naive datetime, assume UTC for display purposes
                    doc_copy[key] = value.replace(tzinfo=UTC).strftime("%Y-%m-%d %H:%M:%S UTC")
                else:
                    doc_copy[key] = value.astimezone(UTC).strftime("%Y-%m-%d %H:%M:%S UTC")
            # Truncate long strings for display purposes, especially IDs
            if isinstance(value, str) and len(value) > 20 and '_id' not in key.lower():
                doc_copy[key] = value[:17] + "..."
            elif isinstance(value, str) and len(value) > 10 and 'id' in key.lower() and key != '_id':
                doc_copy[key] = value[:7] + "..."
        print(doc_copy)
    print("-" * (len(title) + 8))

# --- Course Enrollment Statistics ---

def get_enrollments_per_course():
    """
    Counts total enrollments for each course.
    """
    print("\n--- Course Enrollment Statistics: Enrollments per Course ---")
    pipeline = [
        {"$group": {"_id": "$course_id", "total_enrollments": {"$sum": 1}}},
        {
            "$lookup": {
                "from": "courses",
                "localField": "_id",
                "foreignField": "_id",
                "as": "course_info"
            }
        },
        {"$unwind": "$course_info"},
        {
            "$project": {
                "_id": 0,
                "course_id": "$_id",
                "course_title": "$course_info.title",
                "total_enrollments": 1
            }
        },
        {"$sort": {"total_enrollments": -1}}
    ]
    return list(db.enrollments.aggregate(pipeline))

def get_average_grade_per_course():
    """
    Calculates the average submission grade for each course.
    This involves joining submissions -> assignments -> lessons -> courses.
    """
    print("\n--- Course Enrollment Statistics: Average Grade per Course ---")
    pipeline = [
        # 1. Join submissions with assignments to get lesson_id
        {"$lookup": {
            "from": "assignments",
            "localField": "assignment_id",
            "foreignField": "_id",
            "as": "assignment_info"
        }},
        {"$unwind": "$assignment_info"},
        # 2. Join assignments with lessons to get course_id
        {"$lookup": {
            "from": "lessons",
            "localField": "assignment_info.lesson_id",
            "foreignField": "_id",
            "as": "lesson_info"
        }},
        {"$unwind": "$lesson_info"},
        # 3. Filter out submissions without a grade
        {"$match": {"grade": {"$ne": None}}},
        # 4. Group by course_id and calculate average grade
        {"$group": {
            "_id": "$lesson_info.course_id",
            "average_grade": {"$avg": "$grade"}
        }},
        # 5. Join with courses to get course title
        {"$lookup": {
            "from": "courses",
            "localField": "_id",
            "foreignField": "_id",
            "as": "course_details"
        }},
        {"$unwind": "$course_details"},
        {
            "$project": {
                "_id": 0,
                "course_id": "$_id",
                "course_title": "$course_details.title",
                "average_grade": {"$round": ["$average_grade", 2]} # Round to 2 decimal places
            }
        },
        {"$sort": {"average_grade": -1}}
    ]
    return list(db.submissions.aggregate(pipeline))

def get_course_count_by_category():
    """
    Groups courses by their category and counts the number of courses in each.
    """
    print("\n--- Course Enrollment Statistics: Course Count by Category ---")
    pipeline = [
        {"$group": {"_id": "$category", "total_courses": {"$sum": 1}}},
        {"$project": {"_id": 0, "category": "$_id", "total_courses": 1}},
        {"$sort": {"total_courses": -1}}
    ]
    return list(db.courses.aggregate(pipeline))

# --- Student Performance Analysis ---

def get_average_grade_per_student():
    """
    Calculates the average grade for each student across all their submissions.
    """
    print("\n--- Student Performance Analysis: Average Grade per Student ---")
    pipeline = [
        {"$match": {"grade": {"$ne": None}}}, # Only consider graded submissions
        {"$group": {"_id": "$student_id", "average_grade": {"$avg": "$grade"}}},
        {
            "$lookup": {
                "from": "users",
                "localField": "_id",
                "foreignField": "_id",
                "as": "student_info"
            }
        },
        {"$unwind": "$student_info"},
        {
            "$project": {
                "_id": 0,
                "student_id": "$_id",
                "student_name": {"$concat": ["$student_info.first_name", " ", "$student_info.last_name"]},
                "average_grade": {"$round": ["$average_grade", 2]}
            }
        },
        {"$sort": {"average_grade": -1}}
    ]
    return list(db.submissions.aggregate(pipeline))

def get_course_completion_rate():
    """
    Calculates the completion rate per course based on enrollment status.
    (Assumes 'completed' status in enrollments signifies completion).
    """
    print("\n--- Student Performance Analysis: Course Completion Rate ---")
    pipeline = [
        {"$group": {
            "_id": "$course_id",
            "total_enrollments": {"$sum": 1},
            "completed_enrollments": {
                "$sum": {"$cond": [{"$eq": ["$status", "completed"]}, 1, 0]}
            }
        }},
        {
            "$addFields": {
                "completion_rate": {
                    "$cond": [
                        {"$eq": ["$total_enrollments", 0]},
                        0,
                        {"$multiply": [{"$divide": ["$completed_enrollments", "$total_enrollments"]}, 100]}
                    ]
                }
            }
        },
        {
            "$lookup": {
                "from": "courses",
                "localField": "_id",
                "foreignField": "_id",
                "as": "course_info"
            }
        },
        {"$unwind": "$course_info"},
        {
            "$project": {
                "_id": 0,
                "course_id": "$_id",
                "course_title": "$course_info.title",
                "total_enrollments": 1,
                "completed_enrollments": 1,
                "completion_rate": {"$round": ["$completion_rate", 2]}
            }
        },
        {"$sort": {"completion_rate": -1}}
    ]
    return list(db.enrollments.aggregate(pipeline))

def get_top_performing_students(limit=5):
    """
    Finds the top N students based on their average assignment grades.
    """
    print(f"\n--- Student Performance Analysis: Top {limit} Performing Students ---")
    pipeline = [
        {"$match": {"grade": {"$ne": None}}}, # Only consider graded submissions
        {"$group": {"_id": "$student_id", "average_grade": {"$avg": "$grade"}}},
        {"$sort": {"average_grade": -1}}, # Sort descending by average grade
        {"$limit": limit}, # Get top N
        {
            "$lookup": {
                "from": "users",
                "localField": "_id",
                "foreignField": "_id",
                "as": "student_info"
            }
        },
        {"$unwind": "$student_info"},
        {
            "$project": {
                "_id": 0,
                "student_id": "$_id",
                "student_name": {"$concat": ["$student_info.first_name", " ", "$student_info.last_name"]},
                "average_grade": {"$round": ["$average_grade", 2]}
            }
        }
    ]
    return list(db.submissions.aggregate(pipeline))

# --- Instructor Analytics ---

def get_total_students_taught_by_instructor():
    """
    Calculates the total number of distinct students taught by each instructor
    across all courses they instruct.
    """
    print("\n--- Instructor Analytics: Total Students Taught by Instructor ---")
    pipeline = [
        # 1. Join courses with enrollments
        {"$lookup": {
            "from": "enrollments",
            "localField": "_id",
            "foreignField": "course_id",
            "as": "enrollments"
        }},
        {"$unwind": "$enrollments"},
        # 2. Group by instructor_id and collect distinct student_ids
        {"$group": {
            "_id": "$instructor_id",
            "distinct_students": {"$addToSet": "$enrollments.student_id"} # Collect unique student IDs
        }},
        # 3. Count the number of distinct students
        {"$addFields": {"total_students_taught": {"$size": "$distinct_students"}}},
        # 4. Join with users to get instructor name
        {"$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "_id",
            "as": "instructor_info"
        }},
        {"$unwind": "$instructor_info"},
        {
            "$project": {
                "_id": 0,
                "instructor_id": "$_id",
                "instructor_name": {"$concat": ["$instructor_info.first_name", " ", "$instructor_info.last_name"]},
                "total_students_taught": 1
            }
        },
        {"$sort": {"total_students_taught": -1}}
    ]
    return list(db.courses.aggregate(pipeline))

def get_average_course_rating_per_instructor():
    """
    Calculates the average grade of submissions for courses taught by each instructor.
    (Using submission grades as a proxy for course rating).
    """
    print("\n--- Instructor Analytics: Average 'Course Rating' per Instructor (via grades) ---")
    pipeline = [
        # 1. Join submissions with assignments
        {"$lookup": {
            "from": "assignments",
            "localField": "assignment_id",
            "foreignField": "_id",
            "as": "assignment_details"
        }},
        {"$unwind": "$assignment_details"},
        # 2. Join assignments with lessons
        {"$lookup": {
            "from": "lessons",
            "localField": "assignment_details.lesson_id",
            "foreignField": "_id",
            "as": "lesson_details"
        }},
        {"$unwind": "$lesson_details"},
        # 3. Join lessons with courses
        {"$lookup": {
            "from": "courses",
            "localField": "lesson_details.course_id",
            "foreignField": "_id",
            "as": "course_details"
        }},
        {"$unwind": "$course_details"},
        # 4. Filter for graded submissions
        {"$match": {"grade": {"$ne": None}}},
        # 5. Group by instructor_id and calculate average grade
        {"$group": {
            "_id": "$course_details.instructor_id",
            "average_grade": {"$avg": "$grade"}
        }},
        # 6. Join with users to get instructor name
        {"$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "_id",
            "as": "instructor_info"
        }},
        {"$unwind": "$instructor_info"},
        {
            "$project": {
                "_id": 0,
                "instructor_id": "$_id",
                "instructor_name": {"$concat": ["$instructor_info.first_name", " ", "$instructor_info.last_name"]},
                "average_course_grade": {"$round": ["$average_grade", 2]}
            }
        },
        {"$sort": {"average_course_grade": -1}}
    ]
    return list(db.submissions.aggregate(pipeline))


def get_revenue_generated_per_instructor():
    """
    Calculates the total revenue generated per instructor based on the price of their courses.
    (Simplified: sums the price of all courses taught by an instructor.
    A more advanced model would track actual payments per enrollment).
    """
    print("\n--- Instructor Analytics: Revenue Generated per Instructor (Simplified) ---")
    pipeline = [
        # 1. Group courses by instructor_id and sum their prices
        {"$group": {
            "_id": "$instructor_id",
            "total_course_value": {"$sum": "$price"}
        }},
        # 2. Join with users to get instructor name
        {"$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "_id",
            "as": "instructor_info"
        }},
        {"$unwind": "$instructor_info"},
        {
            "$project": {
                "_id": 0,
                "instructor_id": "$_id",
                "instructor_name": {"$concat": ["$instructor_info.first_name", " ", "$instructor_info.last_name"]},
                "total_revenue_from_courses": {"$round": ["$total_course_value", 2]}
            }
        },
        {"$sort": {"total_revenue_from_courses": -1}}
    ]
    return list(db.courses.aggregate(pipeline))

# --- Advanced Analytics ---

def get_monthly_enrollment_trends():
    """
    Aggregates enrollments by month and year to show enrollment trends over time.
    """
    print("\n--- Advanced Analytics: Monthly Enrollment Trends ---")
    pipeline = [
        {
            "$group": {
                "_id": {
                    "year": {"$year": "$enrollment_date"},
                    "month": {"$month": "$enrollment_date"}
                },
                "total_enrollments": {"$sum": 1}
            }
        },
        {
            "$project": {
                "_id": 0,
                "year": "$_id.year",
                "month": "$_id.month",
                "enrollment_count": "$total_enrollments"
            }
        },
        {"$sort": {"year": 1, "month": 1}} # Sort chronologically
    ]
    return list(db.enrollments.aggregate(pipeline))

def get_most_popular_course_categories():
    """
    Identifies the most popular course categories based on the number of courses.
    (Could be extended to consider enrollments per category).
    """
    print("\n--- Advanced Analytics: Most Popular Course Categories ---")
    pipeline = [
        {"$group": {"_id": "$category", "course_count": {"$sum": 1}}},
        {"$project": {"_id": 0, "category": "$_id", "course_count": 1}},
        {"$sort": {"course_count": -1}},
        {"$limit": 5} # Top 5 categories
    ]
    return list(db.courses.aggregate(pipeline))

def get_student_engagement_by_submissions():
    """
    Calculates student engagement based on the number of assignment submissions.
    (This is a simplified metric; actual engagement might involve lesson views, forum participation etc.)
    """
    print("\n--- Advanced Analytics: Student Engagement by Submissions ---")
    pipeline = [
        {"$group": {"_id": "$student_id", "total_submissions": {"$sum": 1}}},
        {
            "$lookup": {
                "from": "users",
                "localField": "_id",
                "foreignField": "_id",
                "as": "student_info"
            }
        },
        {"$unwind": "$student_info"},
        {
            "$project": {
                "_id": 0,
                "student_id": "$_id",
                "student_name": {"$concat": ["$student_info.first_name", " ", "$student_info.last_name"]},
                "total_submissions": 1
            }
        },
        {"$sort": {"total_submissions": -1}}
    ]
    return list(db.submissions.aggregate(pipeline))


# --- Example Usage of the functions ---
if __name__ == "__main__":
    print("\n--- Running Aggregation Pipeline Examples ---")

    # Course Enrollment Statistics
    print_documents("Course Enrollment Counts", get_enrollments_per_course())
    print_documents("Average Submission Grade per Course", get_average_grade_per_course())
    print_documents("Course Counts by Category", get_course_count_by_category())

    # Student Performance Analysis
    print_documents("Average Grade per Student", get_average_grade_per_student())
    print_documents("Course Completion Rate", get_course_completion_rate())
    print_documents("Top 5 Performing Students", get_top_performing_students(limit=5))

    # Instructor Analytics
    print_documents("Total Students Taught by Each Instructor", get_total_students_taught_by_instructor())
    print_documents("Average 'Course Rating' per Instructor (via grades)", get_average_course_rating_per_instructor())
    print_documents("Revenue Generated per Instructor (Simplified)", get_revenue_generated_per_instructor())

    # Advanced Analytics
    print_documents("Monthly Enrollment Trends", get_monthly_enrollment_trends())
    print_documents("Most Popular Course Categories (Top 5)", get_most_popular_course_categories())
    print_documents("Student Engagement by Total Submissions", get_student_engagement_by_submissions())

    # Close the connection
    client.close()
    print("\nMongoDB connection closed.")

Connected to MongoDB. Performing Advanced Aggregation Pipelines...

--- Running Aggregation Pipeline Examples ---

--- Course Enrollment Statistics: Enrollments per Course ---

--- Course Enrollment Counts ---
{'total_enrollments': 2, 'course_id': 'd659f71...', 'course_title': 'Mastering Program...'}
{'total_enrollments': 2, 'course_id': 'e75448f...', 'course_title': 'Mastering Science...'}
{'total_enrollments': 2, 'course_id': 'cb56f31...', 'course_title': 'Introduction to A...'}
{'total_enrollments': 2, 'course_id': 'f77dff2...', 'course_title': 'Introduction to D...'}
{'total_enrollments': 2, 'course_id': '1a9893c...', 'course_title': 'Fundamentals of D...'}
{'total_enrollments': 1, 'course_id': '64438f4...', 'course_title': 'Mastering Marketi...'}
{'total_enrollments': 1, 'course_id': '20f458a...', 'course_title': 'Introduction to A...'}
{'total_enrollments': 1, 'course_id': '8859601...', 'course_title': 'Advanced Arts Cou...'}
{'total_enrollments': 1, 'course_id': '6f97a67...', 'c

In [29]:
from pymongo import MongoClient, ASCENDING, DESCENDING

# Establish connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

print("Connected to MongoDB. Creating Indexes...")

# --- 1. Index for User Email Lookup ---
# Create a unique index on the 'email' field in the 'users' collection.
# This ensures fast lookup by email and enforces uniqueness.
try:
    db.users.create_index([("email", ASCENDING)], unique=True)
    print("Index 'email_1' created successfully on 'users' collection (unique).")
except Exception as e:
    print(f"Error creating index on 'users.email': {e}")

# --- 2. Index for Course Search by Title and Category ---
# Create a compound index on 'title' and 'category' fields in the 'courses' collection.
# This will speed up queries that filter or sort by both title and category.
try:
    db.courses.create_index([("title", ASCENDING), ("category", ASCENDING)])
    print("Index 'title_1_category_1' created successfully on 'courses' collection.")
except Exception as e:
    print(f"Error creating index on 'courses.title, category': {e}")

# --- 3. Index for Assignment Queries by Due Date ---
# Create an index on the 'due_date' field in the 'assignments' collection.
# This is crucial for efficient range queries and sorting assignments by their due date.
try:
    db.assignments.create_index([("due_date", ASCENDING)])
    print("Index 'due_date_1' created successfully on 'assignments' collection.")
except Exception as e:
    print(f"Error creating index on 'assignments.due_date': {e}")

# --- 4. Index for Enrollment Queries by Student and Course ---
# Create a compound index on 'student_id' and 'course_id' in the 'enrollments' collection.
# This will optimize queries that need to find enrollments for a specific student in a specific course.
try:
    db.enrollments.create_index([("student_id", ASCENDING), ("course_id", ASCENDING)])
    print("Index 'student_id_1_course_id_1' created successfully on 'enrollments' collection.")
except Exception as e:
    print(f"Error creating index on 'enrollments.student_id, course_id': {e}")

# Close the connection
client.close()
print("\nMongoDB connection closed.")

Connected to MongoDB. Creating Indexes...
Error creating index on 'users.email': Index build failed: 0f544e55-3a6a-40ef-abee-57609c62fd74: Collection eduhub_db.users ( 511ae029-2ee6-4184-9b90-54d3afb99480 ) :: caused by :: E11000 duplicate key error collection: eduhub_db.users index: email_1 dup key: { email: "galore.jones@example.com" }, full error: {'ok': 0.0, 'errmsg': 'Index build failed: 0f544e55-3a6a-40ef-abee-57609c62fd74: Collection eduhub_db.users ( 511ae029-2ee6-4184-9b90-54d3afb99480 ) :: caused by :: E11000 duplicate key error collection: eduhub_db.users index: email_1 dup key: { email: "galore.jones@example.com" }', 'code': 11000, 'codeName': 'DuplicateKey', 'keyPattern': {'email': 1}, 'keyValue': {'email': 'galore.jones@example.com'}}
Index 'title_1_category_1' created successfully on 'courses' collection.
Index 'due_date_1' created successfully on 'assignments' collection.
Index 'student_id_1_course_id_1' created successfully on 'enrollments' collection.

MongoDB connect

In [31]:
from pymongo import MongoClient, ASCENDING
from datetime import datetime, timedelta, UTC
import time # For timing function execution
import re # For regex.escape

# Establish connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

print("Connected to MongoDB. Analyzing Query Performance...")

# --- Helper function to display documents cleanly (reused from previous scripts) ---
def print_documents(title, documents, limit=5):
    print(f"\n--- {title} ---")
    if not documents:
        print("No documents found.")
        return
    count = 0
    for doc in documents:
        if doc is None:
            print("  [Document not found or deleted]")
            continue
        
        # Create a copy to avoid modifying the original doc during printing
        doc_copy = doc.copy()
        # Format datetimes for better readability
        for key, value in doc_copy.items():
            if isinstance(value, datetime):
                if value.tzinfo is None:
                    doc_copy[key] = value.replace(tzinfo=UTC).strftime("%Y-%m-%d %H:%M:%S UTC")
                else:
                    doc_copy[key] = value.astimezone(UTC).strftime("%Y-%m-%d %H:%M:%S UTC")
            # Truncate long strings for display purposes, especially IDs
            if isinstance(value, str) and len(value) > 20 and '_id' not in key.lower():
                doc_copy[key] = value[:17] + "..."
            elif isinstance(value, str) and len(value) > 10 and 'id' in key.lower() and key != '_id':
                doc_copy[key] = value[:7] + "..."
        print(doc_copy)
        count += 1
        if count >= limit: # Limit printing for brevity
            print(f"... (showing first {limit} documents, if more exist)")
            break
    print("-" * (len(title) + 8))

# --- Function to run query, explain, and time it ---
def analyze_query(query_func, *args, **kwargs):
    """
    Runs a given query function, measures its execution time,
    and prints its explain() plan and performance metrics.
    """
    print(f"\n--- Analyzing Query: {query_func.__name__} ---")

    # Get the PyMongo cursor or aggregation pipeline cursor
    # We need to construct the query object directly to call explain() on it.
    # This requires some re-structuring from the previous helper functions.

    # Start timer for query execution
    start_time = time.perf_counter()
    
    # Execute the query to get results
    if query_func.__name__ == "find_courses_by_price_range_raw":
        results_cursor = db.courses.find(kwargs["filter"]).explain()
    elif query_func.__name__ == "get_users_joined_last_n_months_raw":
        results_cursor = db.users.find(kwargs["filter"]).explain()
    elif query_func.__name__ == "find_students_in_course_raw":
        # Aggregation pipelines require explain on the aggregate method itself
        results_cursor = db.enrollments.aggregate(kwargs["pipeline"]).explain()
    else:
        # For other types of queries, you'd need to adapt this or pass the cursor
        results_cursor = None
        print("Explain not implemented for this query type in analysis.")


    end_time = time.perf_counter()
    duration_ms = (end_time - start_time) * 1000

    print(f"Query Execution Time: {duration_ms:.2f} ms")

    if results_cursor:
        # For explain() with all_plans_execution mode, total execution time is directly available
        # Otherwise, for simpler explain(), it might be in 'executionStats.executionTimeMillis'
        # Let's try to get executionStats if available.
        if 'executionStats' in results_cursor:
            stats = results_cursor['executionStats']
            print("\n--- Explain() Execution Stats ---")
            print(f"Execution Time (MongoDB): {stats.get('executionTimeMillis', 'N/A')} ms")
            print(f"Documents Examined: {stats.get('totalDocsExamined', 'N/A')}")
            print(f"Keys Examined: {stats.get('totalKeysExamined', 'N/A')}")
            print(f"Used Index: {stats.get('winningPlan', {}).get('inputStage', {}).get('indexName', 'No Index')}")
            print("---------------------------------")
        else:
            print("\n--- Full Explain() Output ---")
            print(results_cursor) # Print full explain output if executionStats not direct
            print("-----------------------------")

    return results_cursor # Return the full explain output for inspection

# --- Optimized Query 1: Find courses with price between $50 and $200 ---
# The previous find_courses_by_price_range already uses efficient $gte/$lte.
# The optimization here relies on the 'price' index.
# (Note: A 'price' index was not explicitly asked for in 5.1 but is crucial here.
# Assuming it exists or would be added for this optimization.)
# Let's define a raw query for explain() to work directly.
def find_courses_by_price_range_raw(min_price, max_price):
    return {"price": {"$gte": min_price, "$lte": max_price}}

# --- Optimized Query 2: Get users who joined in the last 6 months ---
# The previous get_users_joined_last_n_months already uses $gte on 'created_at'.
# The optimization here relies on the 'created_at' index.
# Let's define a raw query for explain() to work directly.
def get_users_joined_last_n_months_raw(num_months):
    six_months_ago = datetime.now(UTC) - timedelta(days=num_months * 30)
    return {"created_at": {"$gte": six_months_ago}}

# Ensure index on created_at for users
try:
    db.users.create_index([("created_at", ASCENDING)])
    print("Index 'created_at_1' created successfully on 'users' collection for date queries.")
except Exception as e:
    print(f"Error creating index on 'users.created_at': {e}")


# --- Optimized Query 3: Find students enrolled in a particular course ---
# This query involves a lookup, which benefits from indexes on the joined fields.
# We'll use the existing pipeline structure for explain().
def find_students_in_course_raw_pipeline(course_title):
    # First, find the course_id for the given title (case-insensitive)
    escaped_course_title = re.escape(course_title)
    course_doc = db.courses.find_one({"title": {"$regex": f"^{escaped_course_title}$", "$options": "i"}})
    
    if not course_doc:
        print(f"Error: Course '{course_title}' not found.")
        return []
    
    course_id = course_doc["_id"]
    
    pipeline = [
        {"$match": {"course_id": course_id}},
        {
            "$lookup": {
                "from": "users",
                "localField": "student_id",
                "foreignField": "_id",
                "as": "student_info"
            }
        },
        {"$unwind": "$student_info"},
        {
            "$project": {
                "_id": "$student_info._id",
                "username": "$student_info.username",
                "email": "$student_info.email",
                "first_name": "$student_info.first_name",
                "last_name": "$student_info.last_name",
                "enrollment_date": "$enrollment_date",
                "enrollment_status": "$status"
            }
        }
    ]
    return pipeline

# --- Main Execution for Analysis ---
if __name__ == "__main__":
    print("\n--- Running Query Optimization Analysis ---")

    # --- Query 1 Analysis: Courses by Price Range ---
    min_price, max_price = 50, 200
    print(f"\n--- Query 1: Courses with Price between ${min_price} and ${max_price} ---")
    
    # Create an index on 'price' for this query to be optimized
    try:
        db.courses.create_index([("price", ASCENDING)])
        print("Index 'price_1' created successfully on 'courses' collection for price range queries.")
    except Exception as e:
        print(f"Error creating index on 'courses.price': {e}")

    # Explain and time the query
    query_filter_price = find_courses_by_price_range_raw(min_price, max_price)
    explain_output_price = analyze_query(find_courses_by_price_range_raw, filter=query_filter_price)
    
    # Run the actual query to see results
    actual_results_price = list(db.courses.find(query_filter_price))
    print_documents("Sample Results (Query 1)", actual_results_price)
    print(f"Total documents found for Query 1: {len(actual_results_price)}")


    # --- Query 2 Analysis: Users Joined in Last 6 Months ---
    num_months = 6
    print(f"\n--- Query 2: Users Joined in Last {num_months} Months ---")

    # Explain and time the query
    query_filter_users = get_users_joined_last_n_months_raw(num_months)
    explain_output_users = analyze_query(get_users_joined_last_n_months_raw, filter=query_filter_users)
    
    # Run the actual query to see results
    actual_results_users = list(db.users.find(query_filter_users))
    print_documents("Sample Results (Query 2)", actual_results_users)
    print(f"Total documents found for Query 2: {len(actual_results_users)}")


    # --- Query 3 Analysis: Students Enrolled in a Specific Course ---
    course_title_for_query = "Introduction to Python Programming" # Or "Introduction to AI Ethics" if you ran create_operations
    print(f"\n--- Query 3: Students Enrolled in '{course_title_for_query}' ---")

    # Make sure we have the necessary indexes for lookups on student_id and course_id
    # These were covered in mongo_db_indexing, but ensuring here.
    try:
        db.enrollments.create_index([("student_id", ASCENDING)])
        db.enrollments.create_index([("course_id", ASCENDING)])
        print("Indexes 'student_id_1' and 'course_id_1' ensured on 'enrollments' for lookups.")
    except Exception as e:
        print(f"Error ensuring enrollment indexes: {e}")

    # Explain and time the query
    pipeline_students = find_students_in_course_raw_pipeline(course_title_for_query)
    # Only proceed if pipeline was successfully generated (i.e., course found)
    if pipeline_students:
        explain_output_students = analyze_query(find_students_in_course_raw, pipeline=pipeline_students)
        
        # Run the actual query to see results
        actual_results_students = list(db.enrollments.aggregate(pipeline_students))
        print_documents("Sample Results (Query 3)", actual_results_students)
        print(f"Total documents found for Query 3: {len(actual_results_students)}")
    else:
        print("Skipping Query 3 analysis as the target course was not found.")

    # Close the connection
    client.close()
    print("\nMongoDB connection closed.")

Connected to MongoDB. Analyzing Query Performance...
Index 'created_at_1' created successfully on 'users' collection for date queries.

--- Running Query Optimization Analysis ---

--- Query 1: Courses with Price between $50 and $200 ---
Index 'price_1' created successfully on 'courses' collection for price range queries.

--- Analyzing Query: find_courses_by_price_range_raw ---
Query Execution Time: 15.36 ms

--- Explain() Execution Stats ---
Execution Time (MongoDB): 12 ms
Documents Examined: 4
Keys Examined: 4
Used Index: No Index
---------------------------------

--- Sample Results (Query 1) ---
{'_id': '20f458a6-72e1-4281-837e-2db3db73a8da', 'title': 'Introduction to A...', 'description': 'This course explo...', 'instructor_id': '8c41e90...', 'category': 'AI & Ethics', 'level': 'intermediate', 'duration': 30, 'price': 99.99, 'tags': ['AI', 'Ethics', 'Philosophy', 'Technology'], 'created_at': '2025-06-14 15:55:49 UTC', 'updated_at': '2025-06-14 15:55:49 UTC', 'is_published': True}

In [33]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, WriteError # Import specific error types
from datetime import datetime, UTC
import uuid
import random

# Establish connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

print("Connected to MongoDB. Demonstrating Error Handling...")

# --- Helper function to display documents cleanly (reused for consistency) ---
def print_documents(title, documents, limit=5):
    print(f"\n--- {title} ---")
    if not documents:
        print("No documents found.")
        return
    count = 0
    for doc in documents:
        if doc is None:
            print("  [Document not found or deleted]")
            continue
        
        doc_copy = doc.copy()
        for key, value in doc_copy.items():
            if isinstance(value, datetime):
                if value.tzinfo is None:
                    doc_copy[key] = value.replace(tzinfo=UTC).strftime("%Y-%m-%d %H:%M:%S UTC")
                else:
                    doc_copy[key] = value.astimezone(UTC).strftime("%Y-%m-%d %H:%M:%S UTC")
            if isinstance(value, str) and len(value) > 20 and '_id' not in key.lower():
                doc_copy[key] = value[:17] + "..."
            elif isinstance(value, str) and len(value) > 10 and 'id' in key.lower() and key != '_id':
                doc_copy[key] = value[:7] + "..."
        print(doc_copy)
        count += 1
        if count >= limit:
            print(f"... (showing first {limit} documents, if more exist)")
            break
    print("-" * (len(title) + 8))

# --- Error Handling Example 1: Duplicate Key Errors ---
def attempt_insert_duplicate_user(user_data):
    """
    Attempts to insert a user document that might cause a DuplicateKeyError
    due to unique constraints on _id or email.
    """
    print(f"\n--- Attempting to insert user (possible duplicate): {user_data.get('email')} ---")
    try:
        result = db.users.insert_one(user_data)
        print(f"Successfully inserted user with ID: {result.inserted_id}")
    except DuplicateKeyError as e:
        print(f"Error: Duplicate Key Error encountered! Details: {e}")
        print("This typically means a user with the same _id or email already exists.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# --- Error Handling Example 2: Invalid Data Type Insertions (Schema Validation) ---
def attempt_insert_invalid_data_user(user_data):
    """
    Attempts to insert a user document with an invalid data type for a field,
    which should trigger a schema validation error (WriteError).
    """
    print(f"\n--- Attempting to insert user with invalid data type: {user_data.get('email')} ---")
    try:
        result = db.users.insert_one(user_data)
        print(f"Successfully inserted user with ID: {result.inserted_id} (Unexpectedly - schema validation might be off or data passed validation)")
    except WriteError as e:
        print(f"Error: Write Error (Invalid Data Type) encountered! Details: {e}")
        print("This typically means a field value did not match the defined bsonType in the schema validator.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# --- Error Handling Example 3: Missing Required Fields (Schema Validation) ---
def attempt_insert_missing_fields_user(user_data):
    """
    Attempts to insert a user document missing a required field,
    which should trigger a schema validation error (WriteError).
    """
    print(f"\n--- Attempting to insert user with missing required fields: {user_data.get('email', 'N/A')} ---")
    try:
        result = db.users.insert_one(user_data)
        print(f"Successfully inserted user with ID: {result.inserted_id} (Unexpectedly - schema validation might be off or data passed validation)")
    except WriteError as e:
        print(f"Error: Write Error (Missing Required Field) encountered! Details: {e}")
        print("This typically means a required field was not provided in the document.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Example Usage of Error Handling Functions ---
if __name__ == "__main__":
    # Ensure there's at least one user for duplicate testing
    existing_user = db.users.find_one({"role": "student"})
    if not existing_user:
        # If no user exists, insert one for test purposes
        test_user_id = str(uuid.uuid4())
        test_user_email = "test.user@example.com"
        db.users.insert_one({
            "_id": test_user_id,
            "username": "testuser",
            "email": test_user_email,
            "password_hash": "testpass",
            "role": "student",
            "created_at": datetime.now(UTC),
            "updated_at": datetime.now(UTC),
            "first_name": "Test",
            "last_name": "User",
            "profile": {"bio": "", "avatar": "", "skills": []},
            "is_active": True
        })
        existing_user = db.users.find_one({"_id": test_user_id})
        print(f"Created a test user: {existing_user['email']} (ID: {existing_user['_id'][:8]})")
    else:
        test_user_id = existing_user["_id"]
        test_user_email = existing_user["email"]
        print(f"Using existing user for tests: {existing_user['email']} (ID: {existing_user['_id'][:8]})")


    # --- Test Case 1: Duplicate Key Error (Existing Email) ---
    duplicate_email_user_data = {
        "_id": str(uuid.uuid4()), # New ID, but duplicate email
        "username": "duplicate_email",
        "email": test_user_email, # This email already exists
        "password_hash": "some_hash",
        "role": "student",
        "created_at": datetime.now(UTC),
        "updated_at": datetime.now(UTC),
        "first_name": "Dup",
        "last_name": "Email",
        "profile": {},
        "is_active": True
    }
    attempt_insert_duplicate_user(duplicate_email_user_data)

    # --- Test Case 2: Duplicate Key Error (Existing _id) ---
    duplicate_id_user_data = {
        "_id": test_user_id, # This _id already exists
        "username": "duplicate_id",
        "email": "another.unique@example.com", # Unique email
        "password_hash": "some_hash",
        "role": "student",
        "created_at": datetime.now(UTC),
        "updated_at": datetime.now(UTC),
        "first_name": "Dup",
        "last_name": "ID",
        "profile": {},
        "is_active": True
    }
    attempt_insert_duplicate_user(duplicate_id_user_data)


    # --- Test Case 3: Invalid Data Type Insertion (Email as int) ---
    invalid_email_user_data = {
        "_id": str(uuid.uuid4()),
        "username": "invalid_data_test",
        "email": 12345, # Invalid: should be string
        "password_hash": "hash",
        "role": "student",
        "created_at": datetime.now(UTC),
        "updated_at": datetime.now(UTC),
        "first_name": "Invalid",
        "last_name": "Type",
        "profile": {},
        "is_active": True
    }
    attempt_insert_invalid_data_user(invalid_email_user_data)

    # --- Test Case 4: Missing Required Field (missing 'username') ---
    missing_username_user_data = {
        "_id": str(uuid.uuid4()),
        "email": "missing.username@example.com",
        "password_hash": "hash",
        "role": "student",
        "created_at": datetime.now(UTC),
        "updated_at": datetime.now(UTC),
        "first_name": "Missing",
        "last_name": "Field",
        "profile": {},
        "is_active": True
    }
    attempt_insert_missing_fields_user(missing_username_user_data)

    # --- Test Case 5: Invalid Enum Value (role as 'manager') ---
    invalid_enum_user_data = {
        "_id": str(uuid.uuid4()),
        "username": "invalid_role",
        "email": "invalid.role@example.com",
        "password_hash": "hash",
        "role": "manager", # Invalid: should be 'student' or 'instructor'
        "created_at": datetime.now(UTC),
        "updated_at": datetime.now(UTC),
        "first_name": "Invalid",
        "last_name": "Enum",
        "profile": {},
        "is_active": True
    }
    attempt_insert_invalid_data_user(invalid_enum_user_data) # Uses invalid_data_type function as it's a schema validation error


    # Close the connection
    client.close()
    print("\nMongoDB connection closed.")

Connected to MongoDB. Demonstrating Error Handling...
Using existing user for tests: grace.anderson0@example.com (ID: 32c92c75)

--- Attempting to insert user (possible duplicate): grace.anderson0@example.com ---
Successfully inserted user with ID: 28245586-2a7b-4ae4-867f-892efce3e38c

--- Attempting to insert user (possible duplicate): another.unique@example.com ---
Error: Duplicate Key Error encountered! Details: E11000 duplicate key error collection: eduhub_db.users index: _id_ dup key: { _id: "32c92c75-cd8c-4661-9777-56905eb7431a" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: eduhub_db.users index: _id_ dup key: { _id: "32c92c75-cd8c-4661-9777-56905eb7431a" }', 'keyPattern': {'_id': 1}, 'keyValue': {'_id': '32c92c75-cd8c-4661-9777-56905eb7431a'}}
This typically means a user with the same _id or email already exists.

--- Attempting to insert user with invalid data type: 12345 ---
Error: Write Error (Invalid Data Type) encountered! Deta